# 정리

과제는 어떤 승객이 대체 차원으로 이동되었는지 **분류**하는 것을 목적으로 한다. 

- 커널 순서
  * train data 탐색 > test data 탐색 > EDA(연속형범주형 변수 분포 확인) > 데이터 전처리 > Modeling > Submission

# 패키지 정리

In [1]:
import plotly.express as px

## plotly.express
---
- cufflinks보다 다양한 차트를 제공하고 사용방법은 seaborn과 비슷하다.

In [ ]:
train_age=train.copy()
test_age=test.copy()

train_age['type']='Train'
test_age['type']='Test'

ageDF=pd.concat([train_age,test_age])
fig=px.histogram(data_frame=ageDF,
                x='Age', color='type', color_discrete_sequence=['#58D68D','#DE3136'],
                marginal='box',nbins=100, template='plotly_white')

fig.update_layout(title='Distribution of Age', title_x=0.5)
fig.show()

![nn](./px.png)

> [express 참고](https://dailyheumsi.tistory.com/118)

## plotly.graph_objects
---
라인 별로 그래프를 할당한다.

In [ ]:
fig = make_subplots(rows=1, cols=2, column_titles=['Train Data', 'Test Data'], x_title='Missing Values')

## train columns 표시
fig.add_trace(go.Bar(x=train_null[0], y=train_null.index, orientation='h',
                     marker=dict(color=[n for n in range(12)],
                     line_color='rgb(0,0,0)',
                     line_width=2,
                     coloraxis='coloraxis')), 1,1)

## test columns 표시
fig.add_trace(go.Bar(x=test_null[0], y=test_null.index, orientation='h',
                    marker=dict(color=[n for n in range(12)],
                                line_color='rgb(0,0,0)',
                                line_width=2,
                                coloraxis='coloraxis')),1,2)

fig.update_layout(showlegend=False, title_text='Column wise Null Value Distribution', title_x=0.5)

![nn](./go.png)

## (from) sklearn.impute (import) SimpleImputer
---
데이터 셋의 missing value를 특정한 값으로 채우는 기능을 제공한다.  
같은 기능을 제공하는 pandas의 fillna()가 더 많이 사용되지만, <u>missing value feature 데이터 셋이 많을 경우</u> SimpleImputer를 사용하는게 코드를 더 간결하게 한다.  
(다만, 다른 특성(features)을 참조해 대푯값을 더 섬세하게 채울수 있다면 fillna()를 사용하는 것이 좋다.)

* fit()으로 strategy에 따른 값을 계산하고, transform()으로 missing 영역을 계산된 값을 채워준다.
* fit()과 transform()의 대상이 같은 경우엔 fit_transform()으로 바로 처리할 수 있다.

>[simpleImputer 참고](https://blog.naver.com/PostView.naver?blogId=seokcrew&logNo=221603139044&parentCategoryNo=42&categoryNo=&viewDate=&isShowPopularPosts=true&from=search)

In [ ]:
imputer_cols = ['Age','FoodCourt','ShoppingMall','Spa','VRDeck','RoomService']
imputer = SimpleImputer(strategy=Strategy)
imputer.fit(train[imputer_cols])

train[imputer_cols]=imputer.transform(train[imputer_cols])
test[imputer_cols]=imputer.transform(test[imputer_cols])

train['HomePlanet'].fillna('Z', inplace=True)
test['HomePlanet'].fillna('Z', inplace=True)

## (from) lazypredict.Supervised (import) LazyClassifier
---
* LazyPredict
 - 많은 모델들의 결과를 한 번에 도출한다.
 - 튜닝하지 않은 Basic 모델을 이용할 때 어떤 모델이 더 잘 작동하는지 이해하는데 도움을 준다. 
 - 데이터가 어떤 모델을 사용했을때, 좋은 결과를 얻어낼 수 있을지 확인이 필요한 경우 사용하면 좋다  
 (튜닝이 없기에 해당 결과를 맹신하는 것은 좋지 않다.)

In [ ]:
clf = LazyClassifier(verbose=0, 
                    ignore_warnings=True,
                    custom_metric=None,
                    predictions=False,
                    random_state=12)

models, predictions = clf.fit(X_train, X_test, y_train, y_test)

![nn](./lazypredict.png)